In [1]:
# load 

import sys
import glob
sys.path.append('/Users/shanxiafeng/Documents/Project/Research/fnirs-prognosis/code/fnirs-treatment-response-prediction')

import time
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import cross_val_score,train_test_split
from utils.utils_mine import*
import tensorflow as tf
import tensorflow.keras as keras
from datetime import date
import numpy as np
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat
from scipy.stats import mannwhitneyu
from statsmodels.stats.multitest import multipletests
import pingouin as pg
import subprocess
import collections

# path of data 


def read_from_file(example_path): # Open the file and read through the first few lines to find where the data starts
    with open(example_path, 'r') as file:
        lines = file.readlines()
        data_start_line = None
        for i, line in enumerate(lines):
            if 'Data' in line:  # This should match the unique identifier of the data section
                data_start_line = i + 1
                # print(lines[data_start_line])
                break

    if data_start_line is not None:

        # Read the data section, skipping the lines of the metadata
        data = pd.read_csv(example_path, skiprows=data_start_line)

        # Now you have metadata and data as separate DataFrames
        # print(data)
    else:
        print("Data section not found.")
        
    np_data = data.to_numpy()
    ch_data = np_data[:, 1:1+52]

    return ch_data

def get_file_name(path, rest):
    file_pattern = os.path.join(path, rest)
    file_list = glob.glob(file_pattern)
    return file_list

def check_replicate_subject(all_subject):
    all_subject_index = [i[3:] for i in all_subject]
    all_subject_index = np.array(all_subject_index).astype(int)
    print(f'all_subject_index -> {all_subject_index}')
    print('all_subject_index.shape -> ', all_subject_index.shape)
    print('len(np.unique(all_subject_index)) -> ', len(np.unique(all_subject_index)))
    is_replicated = len(np.unique(all_subject_index)) != len(all_subject_index)
    print(f"Is there any replicated number in all_subject_index? {is_replicated}")
    if is_replicated:
        replicated_elements = [item for item, count in collections.Counter(all_subject_index).items() if count > 1]
        replicated_indices = np.where(np.isin(all_subject_index, replicated_elements))[0]
        print(f" Element {replicated_elements} shows up in the following indices: {replicated_indices}")
    else: return None
    print(f' now will return replicated_indices[0::2]')
    return replicated_indices[0::2]


follow_up_fold = '/Users/shanxiafeng/Documents/Project/Research/fnirs-prognosis/code/fnirs-treatment-response-prediction/allData/RawData'
T8_path = follow_up_fold + '/T8_fnirs/Session 2_VFT'
base_patient_path = follow_up_fold + '/Baseline_fnirs/Patients'
cli_path = '/Users/shanxiafeng/Documents/Project/Research/fnirs-prognosis/code/fnirs-treatment-response-prediction/allData/fNIRS x MDD Data_Demographics_Clinical.xlsx'

cgi_sgs_data = pd.read_excel(cli_path, sheet_name='SDS_CGI_All Timepoints')

# cgi_sgs_data.iloc[:, 1:7]

excel_data = pd.read_excel(cli_path, sheet_name='Summary T0T8_fNIRS Analysis')
# cgi_sgs_data = pd.read_excel(cgi_sgs_path, sheet_name='SDS_CGI_All Timepoints')
label_hamd = []
demografic_data = []
baseline_clinical_data = []

def check_existing_subject_in_fnirs_path(path):
    for hb in ['_Oxy.csv', '_Deoxy.csv']:
        tmp = 0
        all_subject = []
        for i in os.listdir(path):
            if i[-len(hb):] == hb:
                subject = i.split(' ')[0]
                all_subject.append(subject)
                file_pattern = os.path.join(path, subject+'*'+hb)
                file_list = glob.glob(file_pattern)
                if len(file_list) < 1:
                    print(file_list)
                tmp+=1
        all_subject.sort()
    return all_subject

all_subject = check_existing_subject_in_fnirs_path(T8_path)
print(f'all_subject -> {len(all_subject)}')

# def get_file_name(path, rest):
#     file_pattern = os.path.join(path, rest)
#     file_list = glob.glob(file_pattern)
#     return file_list
# # according to the subject name of all_subject create array now 

mdd_subject_base_post = np.zeros((len(all_subject), 1251, 52, 2, 2))# []#np.zeros((len(all_subject), 1251, 52, 2)) # time, channel, hbo/hbr
all_involve_subject = []
for sub_index, subject in enumerate(all_subject):
    hamd_of_id_t1 = excel_data[excel_data['Subject ID'] == subject]['HAM-D Questionnaire (T1)'].iloc[0]
    hamd_of_id_t8 = excel_data[excel_data['Subject ID'] == subject]['HAM-D Questionnaire (T8)'].iloc[0]
    demographic = excel_data[excel_data['Subject ID'] == subject].iloc[:, 2:13]
    clinical = cgi_sgs_data[cgi_sgs_data['Subject ID'] == subject].iloc[:, 1:7]
    if type(hamd_of_id_t8) is not int:
        print(hamd_of_id_t8)
        continue
    all_involve_subject.append(subject)
    sub_label = [hamd_of_id_t1, hamd_of_id_t8]
    label_hamd.append(sub_label)
    demografic_data.append(demographic)
    baseline_clinical_data.append(clinical)
    for hb_index, hb in enumerate(['_Oxy.csv', '_Deoxy.csv']):

        base_hb_file = get_file_name(base_patient_path, subject+'*'+hb)
        base_hb = read_from_file(base_hb_file[0])
        
        post_hb_file = get_file_name(T8_path, subject+'*'+hb)
        post_hb = read_from_file(post_hb_file[0])
        
        mdd_subject_base_post[sub_index, :, :, hb_index, 0] = base_hb
        mdd_subject_base_post[sub_index, :, :, hb_index, 1] = post_hb
        # print(f"sub :{sub_index} ({subject}) hb: {hb_index} was given base value {np.mean(base_hb)} and post value {np.mean(post_hb)}")

mdd_subject_base = np.array(mdd_subject_base_post)
label_hamd = np.array(label_hamd)
demografic_data = np.squeeze(np.array(demografic_data))
baseline_clinical_data = np.squeeze(np.array(baseline_clinical_data))


# check if there is any replicated subject, becasue there might be two files with same subject names
replicated_indices = check_replicate_subject(all_subject)
print(f'return replicated_indices {replicated_indices}')

print(f'mdd_subject_base -> {mdd_subject_base.shape}')
print(f'label_hamd -> {label_hamd.shape}')
print(f'demografic_data -> {demografic_data.shape}')
print(f'baseline_clinical_data -> {baseline_clinical_data.shape}')
print('all_involve_subject', all_involve_subject)



/Users/shanxiafeng/miniconda3/lib/python3.9/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/Users/shanxiafeng/miniconda3/lib/python3.9/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.11.0 and strictly below 2.14.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.14.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issu

all_subject -> 46
all_subject_index -> [ 2  3  5  6  8  9 10 11 12 13 14 15 16 17 18 19 21 22 23 24 25 26 27 28
 29 30 31 32 33 34 36 37 38 39 40 41 42 43 44 45 46 48 49 50 51 54]
all_subject_index.shape ->  (46,)
len(np.unique(all_subject_index)) ->  46
Is there any replicated number in all_subject_index? False
return replicated_indices None
mdd_subject_base -> (46, 1251, 52, 2, 2)
label_hamd -> (46, 2)
demografic_data -> (46, 11)
baseline_clinical_data -> (46, 6)
all_involve_subject ['PT002', 'PT003', 'PT005', 'PT006', 'PT008', 'PT009', 'PT010', 'PT011', 'PT012', 'PT013', 'PT014', 'PT015', 'PT016', 'PT017', 'PT018', 'PT019', 'PT021', 'PT022', 'PT023', 'PT024', 'PT025', 'PT026', 'PT027', 'PT028', 'PT029', 'PT030', 'PT031', 'PT032', 'PT033', 'PT034', 'PT036', 'PT037', 'PT038', 'PT039', 'PT040', 'PT041', 'PT042', 'PT043', 'PT044', 'PT045', 'PT046', 'PT048', 'PT049', 'PT050', 'PT051', 'PT054']


In [7]:
print('all_subject', len(all_subject))
a = np.reshape(all_subject, (2,-1))

all_subject 46


array([['PT002', 'PT003', 'PT005', 'PT006', 'PT008', 'PT009', 'PT010',
        'PT011', 'PT012', 'PT013', 'PT014', 'PT015', 'PT016', 'PT017',
        'PT018', 'PT019', 'PT021', 'PT022', 'PT023', 'PT024', 'PT025',
        'PT026', 'PT027'],
       ['PT028', 'PT029', 'PT030', 'PT031', 'PT032', 'PT033', 'PT034',
        'PT036', 'PT037', 'PT038', 'PT039', 'PT040', 'PT041', 'PT042',
        'PT043', 'PT044', 'PT045', 'PT046', 'PT048', 'PT049', 'PT050',
        'PT051', 'PT054']], dtype='<U5')

hb_data ->  (64, 52, 2500)


In [ ]:
cgi_sgs_path = '/Users/shanxiafeng/Documents/Project/Research/fnirs-prognosis/code/fnirs-treatment-response-prediction/allData/fNIRS x MDD Data_Demographics_Clinical.xlsx'

cgi_sgs_data = pd.read_excel(cgi_sgs_path, sheet_name='SDS_CGI_All Timepoints')

cgi_sgs_data


In [ ]:
excel_data